# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complete is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Fire up graphlab create

In [7]:
import graphlab
#import graphlab as gl
debug = True

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [8]:
sales = graphlab.SFrame('./kc_house_data.gl/')

# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [9]:
train_data,test_data = sales.random_split(.8,seed=0)

# Useful SFrame summary functions

In order to make use of the closed form solution as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [10]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size() # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 540088.141905
average price via method 2: 540088.141905


As we see we get the same answer both ways

In [11]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print "the sum of price squared is: " + str(sum_prices_squared)

the sum of price squared is: 9.21732513355e+15


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

### Let's take a look at the data, just for reference...

In [12]:
train_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


# Build a generic simple linear regression function 

Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [13]:
def simple_linear_regression(input_feature, output):
    
    size = gl.SArray(input_feature).size()
    
    if(debug) :
        print("----------------------------------------------------------")
        print("  DEBUG: simple_linear_regression()")
        print("----------------------------------------------------------")
        print("Function parameters:")
        print("  Vector X             : ") + str(input_feature)
        print("  Vector Y             : ") + str(output)
        print("  Vector length        : ") + str(size) + "\n"

    # compute the sum of input_feature and output
    sum_in = gl.SArray(input_feature).sum()
    sum_out = gl.SArray(output).sum()
    
    if(debug) :
        print("Intermediate calculations of the regression slope and intercept:")
        print("  Summation of X       : " + str(sum_in))
        print("  Summation of Y       : " + str(sum_out))
    
    # compute the product of the output and the input_feature and its sum
    product_of_in_out = (input_feature * output)
    sum_of_product_of_in_out = product_of_in_out.sum()
    
    if(debug) :
        print("  X * Y                : ") + str(product_of_in_out)
        print("  Summation of [X * Y] : ") + str(sum_of_product_of_in_out)
    
    # compute the squared value of the input_feature and its sum
    square_of_in = gl.SArray() 
    square_of_in = input_feature ** 2
    sum_of_square_of_in = gl.SArray(square_of_in).sum()
    sum_in_squared = sum_in ** 2
    
    if(debug) :
        print("  [X]^2                : ") + str(square_of_in)
        print("  Summation ([X]^2)    : ") + str(sum_of_square_of_in)
        print("  (Summation X)^2      : ") + str(sum_in_squared) + "\n"
                    
    # use the formula for the slope
    slope_numerator = sum_of_product_of_in_out - ((1.0 / size) * (sum_in * sum_out))
    slope_denominator = sum_of_square_of_in - ((1.0 / size) * (sum_in_squared))
    
    if(debug) :
        print("  Compute the slope as : ") + str(slope_numerator) + " / " + str(slope_denominator) + "\n"
    
    slope = slope_numerator / slope_denominator
    
    if(debug) :
        print("SLOPE     : ") + str(slope)
    
    # use the formula for the intercept
    intercept = (sum_out / size) - (slope * ((sum_in / size)))
    
    if(debug) :
        print("INTERCEPT : ") + str(intercept)
        print("----------------------------------------------------------") + "\n"

    return (intercept, slope)

In [14]:
X = graphlab.SArray([0, 1, 2, 3, 4])
Y = graphlab.SArray([1, 3, 7, 13, 21])

(test_intercept, test_slope) = simple_linear_regression(X, Y)

----------------------------------------------------------
  DEBUG: simple_linear_regression()
----------------------------------------------------------
Function parameters:
  Vector X             : [0, 1, 2, 3, 4]
  Vector Y             : [1, 3, 7, 13, 21]
  Vector length        : 5

Intermediate calculations of the regression slope and intercept:
  Summation of X       : 10
  Summation of Y       : 45
  X * Y                : [0, 3, 14, 39, 84]
  Summation of [X * Y] : 140
  [X]^2                : [0.0, 1.0, 4.0, 9.0, 16.0]
  Summation ([X]^2)    : 30.0
  (Summation X)^2      : 100

  Compute the slope as : 50.0 / 10.0

SLOPE     : 5.0
INTERCEPT : -1.0
----------------------------------------------------------



We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [15]:
test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

----------------------------------------------------------
  DEBUG: simple_linear_regression()
----------------------------------------------------------
Function parameters:
  Vector X             : [0, 1, 2, 3, 4]
  Vector Y             : [1, 2, 3, 4, 5]
  Vector length        : 5

Intermediate calculations of the regression slope and intercept:
  Summation of X       : 10
  Summation of Y       : 15
  X * Y                : [0, 2, 6, 12, 20]
  Summation of [X * Y] : 40
  [X]^2                : [0.0, 1.0, 4.0, 9.0, 16.0]
  Summation ([X]^2)    : 30.0
  (Summation X)^2      : 100

  Compute the slope as : 10.0 / 10.0

SLOPE     : 1.0
INTERCEPT : 1.0
----------------------------------------------------------

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [16]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

----------------------------------------------------------
  DEBUG: simple_linear_regression()
----------------------------------------------------------
Function parameters:
  Vector X             : [1180.0, 2570.0, 770.0, 1960.0, 1680.0, 5420.0, 1715.0, 1060.0, 1780.0, 1890.0, 3560.0, 1160.0, 1370.0, 1810.0, 1890.0, 1600.0, 1200.0, 1250.0, 1620.0, 3050.0, 2270.0, 1070.0, 2450.0, 2450.0, 1400.0, 1520.0, 2570.0, 1190.0, 2330.0, 2060.0, 2300.0, 1660.0, 2360.0, 1220.0, 2570.0, 3595.0, 1570.0, 1280.0, 3160.0, 990.0, 2290.0, 1250.0, 2753.0, 1190.0, 3150.0, 1410.0, 1980.0, 2730.0, 2830.0, 2420.0, 3250.0, 1850.0, 2150.0, 2519.0, 1540.0, 1660.0, 2770.0, 2720.0, 2240.0, 1000.0, 3200.0, 4770.0, 1260.0, 2380.0, 3430.0, 1760.0, 1040.0, 1410.0, 3450.0, 2350.0, 2020.0, 1680.0, 960.0, 2140.0, 2660.0, 2770.0, 1610.0, 1030.0, 3520.0, 1200.0, 1580.0, 1580.0, 3300.0, 1160.0, 1810.0, 2320.0, 2070.0, 1980.0, 2190.0, 2920.0, 1210.0, 2340.0, 1670.0, 1240.0, 3140.0, 2310.0, 1260.0, 1540.0, 2080.0, 4380.0, ..

# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [17]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = intercept + slope * input_feature
    
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [18]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [19]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    
    if(debug) :
        print("----------------------------------------------------------")
        print("  DEBUG: get_residual_sum_of_squares()")
        print("----------------------------------------------------------")
        print("Function parameters:")
        print("  input_feature : ") + str(input_feature)
        print("  output        : ") + str(output)
        print("  intercept     : ") + str(intercept)
        print("  slope         : ") + str(slope) + "\n"
    
    # First get the predictions
    predicted_y_hat_vector = get_regression_predictions(input_feature, intercept, slope)
    
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    diff_input_output_vector = predicted_y_hat_vector - output
    residual_vector = diff_input_output_vector ** 2
    
    # square the residuals and add them up
    RSS = residual_vector.sum()

    if(debug) :
        print("Intermediate calculations of the RSS:")
        print("  prediction    : ") + str(diff_input_output_vector)
        print("  residual      : ") + str(residual_vector) + "\n"
        print("RSS : ") + str(RSS)
        print("----------------------------------------------------------") + "\n"

    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [20]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope) # should be 0.0

----------------------------------------------------------
  DEBUG: get_residual_sum_of_squares()
----------------------------------------------------------
Function parameters:
  input_feature : [0, 1, 2, 3, 4]
  output        : [1, 2, 3, 4, 5]
  intercept     : 1.0
  slope         : 1.0

Intermediate calculations of the RSS:
  prediction    : [0.0, 0.0, 0.0, 0.0, 0.0]
  residual      : [0.0, 0.0, 0.0, 0.0, 0.0]

RSS : 0.0
----------------------------------------------------------

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [21]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

----------------------------------------------------------
  DEBUG: get_residual_sum_of_squares()
----------------------------------------------------------
Function parameters:
  input_feature : [1180.0, 2570.0, 770.0, 1960.0, 1680.0, 5420.0, 1715.0, 1060.0, 1780.0, 1890.0, 3560.0, 1160.0, 1370.0, 1810.0, 1890.0, 1600.0, 1200.0, 1250.0, 1620.0, 3050.0, 2270.0, 1070.0, 2450.0, 2450.0, 1400.0, 1520.0, 2570.0, 1190.0, 2330.0, 2060.0, 2300.0, 1660.0, 2360.0, 1220.0, 2570.0, 3595.0, 1570.0, 1280.0, 3160.0, 990.0, 2290.0, 1250.0, 2753.0, 1190.0, 3150.0, 1410.0, 1980.0, 2730.0, 2830.0, 2420.0, 3250.0, 1850.0, 2150.0, 2519.0, 1540.0, 1660.0, 2770.0, 2720.0, 2240.0, 1000.0, 3200.0, 4770.0, 1260.0, 2380.0, 3430.0, 1760.0, 1040.0, 1410.0, 3450.0, 2350.0, 2020.0, 1680.0, 960.0, 2140.0, 2660.0, 2770.0, 1610.0, 1030.0, 3520.0, 1200.0, 1580.0, 1580.0, 3300.0, 1160.0, 1810.0, 2320.0, 2070.0, 1980.0, 2190.0, 2920.0, 1210.0, 2340.0, 1670.0, 1240.0, 3140.0, 2310.0, 1260.0, 1540.0, 2080.0, 4380.0, ... ]


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [22]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    
    if(debug) :
        print("----------------------------------------------------------")
        print("  DEBUG: inverse_regression_predictions()")
        print("----------------------------------------------------------")
        print("Function parameters:")
        print("  output          : ") + str(output)
        print("  intercept       : ") + str(intercept)
        print("  slope           : ") + str(slope) + "\n"
    
    estimated_feature = float((output - intercept)) / slope
    
    if(debug) :
        print("ESTIMATED FEATURE : ") + str(estimated_feature)
        print("----------------------------------------------------------") + "\n"

    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [23]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

----------------------------------------------------------
  DEBUG: inverse_regression_predictions()
----------------------------------------------------------
Function parameters:
  output          : 800000
  intercept       : -47116.0765749
  slope           : 281.958838568

ESTIMATED FEATURE : 3004.39624762
----------------------------------------------------------

The estimated squarefeet for a house worth $800000.00 is 3004


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [24]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
bedrooms_intercept, bedrooms_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)


----------------------------------------------------------
  DEBUG: simple_linear_regression()
----------------------------------------------------------
Function parameters:
  Vector X             : [3.0, 3.0, 2.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 5.0, 3.0, 4.0, 2.0, 3.0, 4.0, 3.0, 5.0, 2.0, 3.0, 3.0, 3.0, 3.0, 4.0, 2.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 5.0, 3.0, 2.0, 5.0, 4.0, 3.0, 5.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 5.0, 5.0, 3.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 3.0, 2.0, 3.0, 3.0, 3.0, 5.0, 3.0, 2.0, 2.0, 5.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, ... ]
  Vector Y             : [221900.0, 538000.0, 180000.0, 604000.0, 510000.0, 1225000.0, 257500.0, 291850.0, 229500.0, 323000.0, 662500.0, 468000.0, 400000.0, 530000.0, 395000.0, 485000.0, 189000.0, 230000.0, 385000.0, 2000000.0, 285000.0, 252700.0, 329000.0, 937000.0, 667000.0, 438000.0, 71900

# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [25]:
# Compute RSS when using bedrooms on TEST data:
RSS_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bedrooms_intercept, bedrooms_slope)
print("RSS of bedroom model : ") + str(RSS_bedrooms)

----------------------------------------------------------
  DEBUG: get_residual_sum_of_squares()
----------------------------------------------------------
Function parameters:
  input_feature : [3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 5.0, 4.0, 3.0, 3.0, 5.0, 3.0, 4.0, 3.0, 5.0, 3.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 2.0, 3.0, 2.0, 4.0, 5.0, 4.0, 3.0, 3.0, 1.0, 3.0, 4.0, 2.0, 5.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 2.0, 3.0, 3.0, 2.0, 4.0, 2.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 1.0, 4.0, 2.0, 4.0, 5.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 5.0, 2.0, 4.0, 3.0, 4.0, 3.0, 4.0, 2.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 2.0, 2.0, ... ]
  output        : [310000.0, 650000.0, 233000.0, 580500.0, 535000.0, 605000.0, 775000.0, 292500.0, 289000.0, 571000.0, 349000.0, 360000.0, 243500.0, 247500.0, 470000.0, 480000.0, 770000.0, 519950.0, 527700.0, 420000.0, 890000.0, 282950.0, 255000.0, 420000.0, 807100.0, 284000.0, 917500.0, 425000.0

In [26]:
# Compute RSS when using squarefeet on TEST data:
RSS_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print("RSS of sqft model : ") + str(RSS_sqft)

----------------------------------------------------------
  DEBUG: get_residual_sum_of_squares()
----------------------------------------------------------
Function parameters:
  input_feature : [1430.0, 2950.0, 1710.0, 2320.0, 1090.0, 2620.0, 4220.0, 2250.0, 1260.0, 2750.0, 1790.0, 1900.0, 1980.0, 1960.0, 2030.0, 3230.0, 2910.0, 1170.0, 2820.0, 1060.0, 2550.0, 1250.0, 2060.0, 2020.0, 2680.0, 1560.0, 3880.0, 1120.0, 1670.0, 1940.0, 910.0, 2480.0, 2440.0, 850.0, 1750.0, 1510.0, 1720.0, 3760.0, 3650.0, 1810.0, 2100.0, 760.0, 1700.0, 2600.0, 860.0, 3830.0, 3500.0, 2420.0, 2160.0, 1970.0, 2040.0, 2200.0, 4290.0, 4290.0, 1470.0, 2030.0, 1450.0, 1240.0, 2050.0, 1070.0, 2490.0, 2930.0, 1420.0, 2500.0, 2580.0, 1390.0, 560.0, 2270.0, 990.0, 1750.0, 3040.0, 3520.0, 2090.0, 2180.0, 1640.0, 2510.0, 2120.0, 1980.0, 1600.0, 3300.0, 2720.0, 940.0, 1820.0, 1510.0, 2390.0, 2910.0, 2200.0, 950.0, 2350.0, 4040.0, 2610.0, 1250.0, 940.0, 1220.0, 1240.0, 2570.0, 1850.0, 2270.0, 1610.0, 630.0, ... ]
  outpu